In [ ]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser 
import bs4
import os  # 환경 변수 설정을 위한 모듈


# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = ""


In [4]:

# 1. 뉴스기사 내용을 웹에서 로드하는 단계
# - WebBaseLoader를 사용하여 특정 웹사이트(네이버 뉴스 기사)에서 텍스트를 가져온다.
# - bs4.SoupStrainer를 사용하여 필요한 부분(div 태그의 특정 클래스)을 선택적으로 파싱한다.
# 조건: 크롤링할 네이버 뉴스 기사를 다른 기사로 바꿔보세요.
# 참고: 에러가 나는 경우 뉴스 기사에서 F12(개발자 도구)를 눌러 기사 제목과 내용이 담긴 올바른 클래스명을 가져오세요.
# 참고: 저작권으로 인해 기사 자체에서 크롤링이 막혀 있을 수도 있습니다. 
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/296/0000087319?sid=103",),  # 크롤링할 네이버 뉴스 링크 입력
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

# 뉴스 기사 내용을 로드하여 문서 형태로 저장
docs = loader.load()
print(f"문서의 수: {len(docs)}")  # 로드된 문서 개수 확인

문서의 수: 1


In [5]:

# 2. 불러온 뉴스 기사를 청크(chunk)로 나누고, 인덱싱
# - RecursiveCharacterTextSplitter를 사용하여 긴 텍스트를 일정 크기의 청크로 나눈다.
# - chunk_size: 한 개의 청크 크기 (문자 수)
# - chunk_overlap: 청크 간 겹치는 부분의 크기 (연속성 유지)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 여러 개의 작은 청크로 나눔
splits = text_splitter.split_documents(docs)
print(f"청크 개수: {len(splits)}")  # 생성된 청크 개수 확인


청크 개수: 4


In [6]:

# 3. 벡터스토어(Vector Store)를 생성
# - FAISS(Facebook AI Similarity Search)를 사용하여 벡터 DB를 생성
# - OpenAIEmbeddings를 사용하여 텍스트를 벡터로 변환
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))


C:\Users\SSAFY\AppData\Local\Temp\ipykernel_10172\1861515578.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))


In [7]:

# 4. 뉴스 기사에서 정보를 검색할 수 있도록 검색기(retriever) 생성
retriever = vectorstore.as_retriever()


In [8]:

# 5. 프롬프트 템플릿 정의
# - 검색된 문맥(Context)과 사용자 질문(Question)을 조합하여 LLM에 입력
# - 문맥에서 답을 찾을 수 없으면 적절한 메시지를 출력
prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)에서 주어진 질문(question)에 답하는 것입니다.
검색된 다음 문맥(context)을 사용하여 질문(question)에 답하세요. 만약, 주어진 문맥(context)에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다`라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)



In [9]:
# 6. LLM 모델 선택
# - OpenAI의 GPT-4o-mini 모델을 사용하여 검색된 정보를 바탕으로 응답을 생성
# - temperature=0 설정으로 일관된 답변을 생성하도록 설정
llm = ChatOpenAI(model="gpt-5-nano", temperature=1)



C:\Users\SSAFY\AppData\Local\Temp\ipykernel_10172\2099748096.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-5-nano", temperature=1)


In [10]:
# 7. RAG(Retrieval-Augmented Generation) 체인 생성
# - 사용자의 질문을 받아 검색(retriever)을 수행한 후, 결과를 프롬프트(prompt)와 결합
# - 결합된 입력을 LLM에 전달하고, 응답을 출력 형식으로 변환
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



In [11]:
# 실행 예시 (사용자가 입력한 질문)
user_question = "이 뉴스에서 다룬 주요 이슈는 무엇인가요?"
response = rag_chain.invoke(user_question)
print(response)  # 최종 생성된 답변 출력

주요 이슈는 씹는 물질의 재질이 뇌의 항산화 물질인 글루타치온 수치와 기억력 등 인지 기능에 어떤 영향을 주는지 여부입니다. 연구에 따르면 나무 막대를 씹은 그룹에서 씹기 전후 글루타치온 수치가 증가했고, 이 증가가 기억력 테스트 점수와 양의 관련성을 보였습니다. 반면 껌을 씹은 그룹은 유의미한 변화가 없었고 두 그룹 간 차이는 통계적으로 유의미하진 않았습니다. 따라서 이 뉴스의 핵심은 단단한 물질을 씹는 행위가 뇌의 항산화 수치와 특정 기억력에 영향을 줄 수 있는지에 대한 초기 연구 결과를 다루고 있다는 점입니다.


In [12]:
# 실행 예시 (사용자가 입력한 질문)
user_question = "뉴스 내용 담겨 있는 질문 넣어보기"
response = rag_chain.invoke(user_question)
print(response)  # 최종 생성된 답변 출력

다음은 주어진 뉴스 내용에 대한 요약과 핵심 포인트입니다.

- 연구 주제: 껌보다 단단한 물질(작은 나무 막대기)을 씹는 것이 뇌의 항산화 물질인 글루타치온 수치와 기억력에 미치는 영향
- 연구 대상 및 설계: 건강한 대학생 52명을 두 그룹으로 나눠 비교. 한 그룹은 표준 파라핀 왁스 껌을 씹고, 다른 그룹은 나무 막대기를 씹음. 씹기는 5분 동안, 1초에 한 번씩 오른쪽 입으로 씹되 30초 씹고 30초 쉬기를 반복.
- 측정 방법: 씹기 전후에 전대상피질의 글루타치온 수치를 자기공명분광법으로 측정하고, 신경심리학적 테스트로 기억, 주의, 언어, 시각-공간 능력을 평가.
- 주요 결과
  - 나무 막대기를 씹은 그룹은 씹기 후 글루타치온 수치가 상당히 증가하는 경향을 보임.
  - 껌을 씹은 그룹은 글루타치온 수치에 유의미한 변화가 없음.
  - 두 그룹 간 글루타치온 수치 변화의 통계적 차이는 유의미하지 않으나, 나무 씹기 그룹의 증가 추세가 더 큼.
  - 나무 씹기 그룹에서 글루타치온 증가가 기억 과제 점수와 긍정적으로 관련됨. 반면 껌 그룹에선 이 연관성이 발견되지 않음.
- 해석 및 시사점: 단단한 재질을 씹는 행위가 뇌의 항산화 수치를 증가시키고 기억력의 특정 측면을 개선하는 데 연결될 수 있음. 다만 이는 특정 나무의 효능이라기보다 씹는 재질의 물리적 특성에 의한 효과일 가능성이 있음.
- 주의점: 연구는 초기 단계이며, 일반화하려면 추가 연구가 필요.


In [13]:
# 실행 예시 (사용자가 입력한 질문)
user_question = "뉴스 내용을 요약하여 영어로 번역해주세요."
response = rag_chain.invoke(user_question)
print(response)  # 최종 생성된 답변 출력

다음은 주어진 문맥의 내용을 한국어로 요약한 모습입니다.

- 연구 주제: 껌보다 단단한 물체를 씹는 것이 뇌의 항산화 물질인 글루타치온 수치와 기억력에 미치는 영향을 알아보는 연구.
- 연구 설계: 경북대 연구진이 건강한 대학생 52명을 두 그룹으로 나눴습니다. 한 그룹은 껌을, 다른 그룹은 팝시클 스틱과 비슷한 작은 나무 막대를 씹었습니다. 씹기는 5분 동안 1초에 한 번 일정하게 하되 30초 씹고 30초 쉬는 패턴으로 진행되었습니다. 씹기 전후로 전대상피질(ACC)에서 글루타치온 수치를 자기공명분광법으로 측정했고, 신경심리 평가도 실시했습니다.
- 주요 결과: 나무 막대를 씹은 그룹에서 씹기 전과 비교해 글루타치온 수치가 유의하게 증가했습니다. 껌을 씹은 그룹에서는 유의한 변화가 없었습니다. 두 그룹 간 차이는 통계적으로 유의하진 않지만, 나무 씹기 그룹이 더 큰 증가 추세를 보였습니다. 또한 나무 씹기 그룹의 글루타치온 증가 정도는 기억 및 스토리 기억 테스트 점수와 양의 관련이 있었습니다(나무 씹기 그룹에서만 관찰). 껌 그룹에선 기억력과 글루타치온 변화 간 연관성이 없었습니다.
- 결론 및 시사점: 딱딱한 재질을 씹는 행위가 뇌의 항산화 수치를 증가시키고 기억력의 특정 측면을 향상시킬 수 있는 가능성을 시사합니다. 다만 특정 나무 종류를 대상으로 한 연구는 아니며, 두 그룹 간 차이가 통계적으로 확실하지 않아 추가 연구가 필요합니다.
- 참고: Frontiers in Systems Neuroscience에 게재된 연구를 PsyPost가 보도했습니다.
